In [17]:
import platform
import io,os
import time
import zipfile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from google.colab import files
from matplotlib.pyplot import cm 

def install_pytorch():
    os = platform.system()
    if os == "Linux":
        !pip3 install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
    elif os == "Windows":
        !pip3 install http://download.pytorch.org/whl/cu90/torch-0.4.0-cp36-cp36m-win_amd64.whl 
    !pip3 install torchvision


# Install PyTorch.
install_pytorch()

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

use_gpu = torch.cuda.is_available()
print("GPU Available: {}".format(use_gpu))

GPU Available: False


In [4]:
# # Load data from local file system.
# van_path = "./vancouver.csv"
# if not os.path.isfile(van_path):
#   print('upload vancouver.csv')
#   uploaded = files.upload()  
# van_module = pd.read_csv(van_path, sep=",")


def read_dir_file(case_f):  # case_f = 0 for uploading one File and case_f = 1 for uploading one Zipped Directory
  uploaded = files.upload()    # to upload a Full Directory, please Zip it first (use WinZip)
  for fn in uploaded.keys():
    name = fn  #.encode('utf-8')
    print('\nfile after encode', name)
    name = io.BytesIO(uploaded[name])
    if case_f == 0:    # case of uploading 'One File only'
      print('\n file name: ', name)
      return name
    else:   # case of uploading a directory and its subdirectories and files
      zfile = zipfile.ZipFile(name, 'r')   # unzip the directory 
      zfile.extractall()
      for d in zfile.namelist():   # d = directory
        print('\n main directory name: ', d)
        return d
  print('Done!')
  
read_dir_file(1)
!ls

Saving toronto2015.zip to toronto2015 (1).zip

file after encode toronto2015.zip

 main directory name:  eng-hourly-01012015-01312015.csv
datalab				  eng-hourly-10012015-10312015.csv
eng-hourly-01012015-01312015.csv  eng-hourly-11012015-11302015.csv
eng-hourly-02012015-02282015.csv  eng-hourly-12012015-12312015.csv
eng-hourly-03012015-03312015.csv  f31103_122_2015-01-01_2015-12-31.csv
eng-hourly-04012015-04302015.csv  f31103_124_2015-01-01_2015-12-31.csv
eng-hourly-05012015-05312015.csv  f31103_35_2015-01-01_2015-12-31.csv
eng-hourly-06012015-06302015.csv  f31103_36_2015-01-01_2015-12-31.csv
eng-hourly-07012015-07312015.csv  f31103_37_2015-01-01_2015-12-31.csv
eng-hourly-08012015-08312015.csv  toronto2015 (1).zip
eng-hourly-09012015-09302015.csv  toronto2015.zip


In [5]:
!ls

datalab				  eng-hourly-10012015-10312015.csv
eng-hourly-01012015-01312015.csv  eng-hourly-11012015-11302015.csv
eng-hourly-02012015-02282015.csv  eng-hourly-12012015-12312015.csv
eng-hourly-03012015-03312015.csv  f31103_122_2015-01-01_2015-12-31.csv
eng-hourly-04012015-04302015.csv  f31103_124_2015-01-01_2015-12-31.csv
eng-hourly-05012015-05312015.csv  f31103_35_2015-01-01_2015-12-31.csv
eng-hourly-06012015-06302015.csv  f31103_36_2015-01-01_2015-12-31.csv
eng-hourly-07012015-07312015.csv  f31103_37_2015-01-01_2015-12-31.csv
eng-hourly-08012015-08312015.csv  toronto2015 (1).zip
eng-hourly-09012015-09302015.csv  toronto2015.zip


In [6]:
trt_w01=pd.read_csv('./eng-hourly-01012015-01312015.csv',sep=',')
trt_w02=pd.read_csv('./eng-hourly-02012015-02282015.csv',sep=',')
trt_w03=pd.read_csv('./eng-hourly-03012015-03312015.csv',sep=',')
trt_w04=pd.read_csv('./eng-hourly-04012015-04302015.csv',sep=',')
trt_w05=pd.read_csv('./eng-hourly-05012015-05312015.csv',sep=',')
trt_w06=pd.read_csv('./eng-hourly-06012015-06302015.csv',sep=',')
trt_w07=pd.read_csv('./eng-hourly-07012015-07312015.csv',sep=',')
trt_w08=pd.read_csv('./eng-hourly-08012015-08312015.csv',sep=',')
trt_w09=pd.read_csv('./eng-hourly-09012015-09302015.csv',sep=',')
trt_w10=pd.read_csv('./eng-hourly-10012015-10312015.csv',sep=',')
trt_w11=pd.read_csv('./eng-hourly-11012015-11302015.csv',sep=',')
trt_w12=pd.read_csv('./eng-hourly-12012015-12312015.csv',sep=',')

weather=pd.concat([trt_w01,trt_w02,trt_w03,trt_w04,trt_w05,trt_w06,trt_w07,trt_w08,trt_w09,trt_w10,trt_w11,trt_w12],ignore_index=True)

weather=pd.DataFrame(weather,columns=
            [weather.columns[2],weather.columns[4],
            weather.columns[5],weather.columns[6],weather.columns[8],
            weather.columns[11],weather.columns[12]])

weather

,Dew Point Temp (°C),Rel Hum (%),Stn Press (kPa),Temp (°C),Visibility (km),Wind Dir (10s deg),Wind Spd (km/h)
0,-13.2,58.0,100.68,-6.2,16.1,25.0,43.0
1,-13.7,55.0,100.59,-6.0,16.1,24.0,41.0
2,-13.3,57.0,100.56,-6.2,16.1,24.0,45.0
3,-12.0,63.0,100.58,-6.0,16.1,24.0,32.0
4,-12.2,62.0,100.50,-6.0,16.1,24.0,35.0
5,-10.9,66.0,100.44,-5.5,16.1,25.0,43.0
6,-10.4,66.0,100.43,-5.0,16.1,24.0,54.0
7,-10.0,68.0,100.44,-4.9,16.1,24.0,46.0
8,-10.4,65.0,100.37,-4.8,16.1,23.0,52.0
9,-9.8,67.0,100.38,-4.5,16.1,24.0,45.0


In [7]:
trt_NO=pd.read_csv('./f31103_35_2015-01-01_2015-12-31.csv',sep=',')
tmp_NO=trt_NO[trt_NO.columns[3:]]
tmp_NO=pd.DataFrame(data=np.array(tmp_NO).reshape(-1))
data=weather.assign(NO=tmp_NO)

trt_NO2=pd.read_csv('./f31103_36_2015-01-01_2015-12-31.csv',sep=',')
tmp_NO2=trt_NO2[trt_NO2.columns[3:]]
tmp_NO2=pd.DataFrame(data=np.array(tmp_NO2).reshape(-1))
data=data.assign(NO2=tmp_NO2)

trt_NOx=pd.read_csv('./f31103_37_2015-01-01_2015-12-31.csv',sep=',')
tmp_NOx=trt_NOx[trt_NOx.columns[3:]]
tmp_NOx=pd.DataFrame(data=np.array(tmp_NOx).reshape(-1))
data=data.assign(NOx=tmp_NOx)

trt_O3=pd.read_csv('./f31103_122_2015-01-01_2015-12-31.csv',sep=',')
tmp_O3=trt_O3[trt_O3.columns[3:]]
tmp_O3=pd.DataFrame(data=np.array(tmp_O3).reshape(-1))
data=data.assign(O3=tmp_O3)

trt_O3=pd.read_csv('./f31103_122_2015-01-01_2015-12-31.csv',sep=',')
tmp_O3=trt_O3[trt_O3.columns[3:]]
tmp_O3=pd.DataFrame(data=np.array(tmp_O3).reshape(-1))
data=data.assign(O3=tmp_O3)

trt_PM25=pd.read_csv('./f31103_124_2015-01-01_2015-12-31.csv',sep=',')
tmp_PM25=trt_PM25[trt_PM25.columns[3:]]
tmp_PM25=pd.DataFrame(data=np.array(tmp_PM25).reshape(-1))
data=data.assign(PM25=tmp_PM25)

data

,Dew Point Temp (°C),Rel Hum (%),Stn Press (kPa),Temp (°C),Visibility (km),Wind Dir (10s deg),Wind Spd (km/h),NO,NO2,NOx,O3,PM25
0,-13.2,58.0,100.68,-6.2,16.1,25.0,43.0,3,15,18,19,6
1,-13.7,55.0,100.59,-6.0,16.1,24.0,41.0,2,14,16,19,7
2,-13.3,57.0,100.56,-6.2,16.1,24.0,45.0,1,10,11,22,7
3,-12.0,63.0,100.58,-6.0,16.1,24.0,32.0,1,10,11,21,7
4,-12.2,62.0,100.50,-6.0,16.1,24.0,35.0,2,11,13,19,7
5,-10.9,66.0,100.44,-5.5,16.1,25.0,43.0,1,10,11,19,6
6,-10.4,66.0,100.43,-5.0,16.1,24.0,54.0,2,9,11,19,6
7,-10.0,68.0,100.44,-4.9,16.1,24.0,46.0,1,7,9,19,6
8,-10.4,65.0,100.37,-4.8,16.1,23.0,52.0,1,7,8,20,6
9,-9.8,67.0,100.38,-4.5,16.1,24.0,45.0,4,9,13,19,6


In [ ]:
# converted_data = []
# for x in data:
#   converted_data.append([float(elm) for elm in x[0:]])

converted_data = np.array(data, dtype = float)  

is_next_nan=False
for y in range(0, converted_data.shape[1]):
  for x in range(0, converted_data.shape[0]-1):
    if(is_next_nan):
        converted_data[x,y] = converted_data[x-1,y]
    if(math.isnan(converted_data[x,y] or converted_data[x,y]==9999 or converted_data[x,y]==-999) and is_next_nan == False):
      j=1
      while(math.isnan(converted_data[x+j,y])):
          j+=1
          is_next_nan=True
      converted_data[x,y] = (converted_data[x+j,y] + converted_data[x-1,y])/2  
      
    if (not(math.isnan(converted_data[x+1,y]))) and is_next_nan:
      is_next_nan = False
      
converted_data

array([[-13.2 ,  58.  , 100.68, ...,  18.  ,  19.  ,   6.  ],
       [-13.7 ,  55.  , 100.59, ...,  16.  ,  19.  ,   7.  ],
       [-13.3 ,  57.  , 100.56, ...,  11.  ,  22.  ,   7.  ],
       ...,
       [ -2.7 ,  74.  , 101.01, ...,  14.  ,  23.  ,  10.  ],
       [ -3.  ,  73.  , 100.98, ...,  15.  ,  22.  ,  10.  ],
       [ -3.4 ,  71.  , 100.92, ...,  16.  ,  21.  ,  11.  ]])

In [20]:
sequence_len = 32+1

tensor_test_data = []
# for x in range(0, len(train_data)):
#   try:
tensor_test_data= torch.FloatTensor(converted_data)
#   catch ValueError:
#     tensor_test_data.append(-1)

print(tensor_test_data)

tensor([[  -13.2000,    58.0000,   100.6800,  ...,    18.0000,
            19.0000,     6.0000],
        [  -13.7000,    55.0000,   100.5900,  ...,    16.0000,
            19.0000,     7.0000],
        [  -13.3000,    57.0000,   100.5600,  ...,    11.0000,
            22.0000,     7.0000],
        ...,
        [   -2.7000,    74.0000,   101.0100,  ...,    14.0000,
            23.0000,    10.0000],
        [   -3.0000,    73.0000,   100.9800,  ...,    15.0000,
            22.0000,    10.0000],
        [   -3.4000,    71.0000,   100.9200,  ...,    16.0000,
            21.0000,    11.0000]])
